In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
from scipy.io import wavfile
from scipy.fftpack import fft,ifft,dct,idct
import pywt

In [2]:
No_of_zeros=2000
N=65536
Sig=[]
rate, x = wavfile.read('audio.wav')
for i in range(N):
    Sig.append(x[i])
Sig=np.array(Sig)

### Discrete Cosine Transform

In [3]:
C=np.array(dct(Sig))
scipy.io.wavfile.write(('dct.wav'),rate,C.astype(Sig.dtype))

In [4]:
def energy_compact_DCT(C,No_of_zeros):
    N=len(C)
    X_ec_dct=[]
    for i in range(N):
        if i<=(N-1-No_of_zeros):
            X_ec_dct.append(C[i])
    return X_ec_dct

In [5]:
def energy_decompact_DCT(N,No_of_zeros):
    r1, a = wavfile.read('compactdct.wav')
    X1=[]
    for i in range(N-No_of_zeros):
        X1.append(a[i])
    X_edc=[]
    for i in range(N):
        if i<=(N-1-No_of_zeros):
            X_edc.append(X1[i])
        else:
            X_edc.append(0)
    return X_edc

In [6]:
X_ec_dct=energy_compact_DCT(C,No_of_zeros)
X_ec_dct=np.array(X_ec_dct)
scipy.io.wavfile.write(('compactdct.wav'),rate,X_ec_dct.astype(Sig.dtype))

In [7]:
X_edc=energy_decompact_DCT(N,No_of_zeros)
x_edc=idct(X_edc)/(2*rate)
x_edc=np.array(x_edc)
scipy.io.wavfile.write(('resultdct.wav'),rate,x_edc.astype(Sig.dtype))

### Discrete Fourier Transform

In [8]:
D=fft(Sig)
D=np.array(D)
r=np.array(Sig)
scipy.io.wavfile.write(('dft.wav'),rate,D.astype(Sig.dtype))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


In [9]:
def energy_compact_DFT(X,No_of_zeros):
    X_ec_dft=[]
    N=len(X)
    for i in range(N):
        if i<=int((N-1-No_of_zeros)/2):
            X_ec_dft.append(X[i])
        elif i>=int((N+1+No_of_zeros)/2) and i<=N-1:
            X_ec_dft.append(X[i])
    return X_ec_dft

In [10]:
def energy_decompact_DFT(N):
    r1, a = wavfile.read('compactdft.wav')
    X1=[]
    for i in range(N-No_of_zeros):
        X1.append(a[i])
    X_edc=[]
    for i in range(N):
        if i<=int((N-1-No_of_zeros)/2):
            X_edc.append(X1[i])
        elif i>=int((N+1-No_of_zeros)/2) and i<=int((N-1+No_of_zeros)/2):
            X_edc.append(0)
        elif i>=int((N+1+No_of_zeros)/2) and i<=N-1:
            p=i-No_of_zeros
            X_edc.append(X1[p])
    return X_edc

In [11]:
X_ec_dft=energy_compact_DFT(D,No_of_zeros)
X_ec_dft=np.array(X_ec_dft)
scipy.io.wavfile.write(('compactdft.wav'),rate,X_ec_dft.astype(Sig.dtype))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
X_edc=energy_decompact_DFT(N)
x_edc=ifft(X_edc)
x_edc=np.array(x_edc)
scipy.io.wavfile.write(('resultdft.wav'),rate,x_edc.astype(Sig.dtype))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


### Haar Transform

In [13]:
def haar(x):
    out = []
    (ca, cd) = pywt.dwt(x,'haar')
    out = list(cd)+out
    while len(ca)!=1:
        (ca, cd) = pywt.dwt(ca,'haar')
        out = list(cd)+out
    out = list(ca)+out
    return out

In [14]:
def ihaar(x):
    N = len(x)
    i = 0
    x[:2] = pywt.idwt([x[0]], [x[1]],'haar')
    i = 2
    while 2*i<N+1:
        try:
            x[:2*i] = pywt.idwt(x[:i],x[i:2*i],'haar')
        except:
            print(i)
            return
        i = 2*i
    return x

In [15]:
H=haar(Sig)
H=np.array(H)
scipy.io.wavfile.write(('haar.wav'),rate,H.astype(Sig.dtype))

In [16]:
def energy_compact_haar(H,No_of_zeros):
    N=len(C)
    X_ec=[]
    for i in range(N):
        if i<=(N-1-No_of_zeros):
            X_ec.append(C[i])
    return X_ec

In [17]:
def energy_decompact_haar(N,No_of_zeros):
    r1, a = wavfile.read('compacthaar.wav')
    X1=[]
    for i in range(N-No_of_zeros):
        X1.append(a[i])
    X_edc=[]
    for i in range(N):
        if i<=(N-1-No_of_zeros):
            X_edc.append(X1[i])
        else:
            X_edc.append(0)
    return X_edc

In [18]:
X_ec=energy_compact_haar(H,No_of_zeros)
X_ec=np.array(X_ec)
scipy.io.wavfile.write(('compacthaar.wav'),rate,X_ec.astype(Sig.dtype))

In [19]:
X_edc=energy_decompact_haar(N,No_of_zeros)
x_edc=ihaar(X_edc)
x_edc=np.array(x_edc)
scipy.io.wavfile.write(('resulthaar.wav'),rate,x_edc.astype(Sig.dtype))